# Load Data

In [2]:
# import modules
import pandas as pd
import numpy as np
import sqlalchemy as sqla
import pickle
from sqlalchemy import create_engine

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import metrics

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

import pymysql
from sshtunnel import SSHTunnelForwarder
from datetime import datetime, timedelta

import spacy

# Load pre-trained word embeddings (e.g., spaCy's medium English model)
nlp = spacy.load("en_core_web_sm")


from shapely.geometry import MultiPolygon, Polygon
from shapely.ops import nearest_points
import geopandas as gpd
from shapely import wkt

In [3]:
ssh_host = '137.43.49.79'
ssh_port = 22
ssh_username = 'student'
ssh_password = 'Ucd-cs-2023!'

mysql_user = 'root'
mysql_password = 'Group@18'
mysql_db = 'smartcityexplorer'

with SSHTunnelForwarder(
    (ssh_host, ssh_port),
    ssh_username=ssh_username,
    ssh_password=ssh_password,
    remote_bind_address=('127.0.0.1', 3306)
) as tunnel:
    conn = pymysql.connect(
        host='127.0.0.1',
        user=mysql_user,
        password=mysql_password,
        db=mysql_db,
        port=tunnel.local_bind_port
    )
    
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_static")
    results = cursor.fetchall()

    # Convert the results to a pandas DataFrame
    import pandas as pd
    df_venue_static = pd.DataFrame(results, columns=[column[0] for column in cursor.description])
    df_rest_static = pd.DataFrame(results, columns=[column[0] for column in cursor.description])
    
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_timings")
    results = cursor.fetchall()
    df_venue_timings = pd.DataFrame(results, columns=[column[0] for column in cursor.description])

    cursor = conn.cursor()
    cursor.execute("SELECT * FROM venue_merged")
    results = cursor.fetchall()
    df_venue_merged = pd.DataFrame(results, columns=[column[0] for column in cursor.description])    
    
    # Close the database connection
    conn.close()

In [4]:
df_manhattan_zone = pd.read_csv('../data_manipulation/taxi_zones.csv')
df_manhattan_zone.head(50)


,Unnamed: 0,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough,Unnamed: 7,Unnamed: 8,...,Unnamed: 6947,Unnamed: 6948,Unnamed: 6949,Unnamed: 6950,Unnamed: 6951,Unnamed: 6952,Unnamed: 6953,Unnamed: 6954,Unnamed: 6955,Unnamed: 6956
0,4,4,0.043567,MULTIPOLYGON (((-73.97177410965318 40.72582128...,0.000112,Alphabet City,4,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,24,0.047000,MULTIPOLYGON (((-73.95953658899997 40.79871852...,0.000061,Bloomingdale,24,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13,12,0.036661,MULTIPOLYGON (((-74.01565756599994 40.70483308...,0.000042,Battery Park,12,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14,13,0.050281,MULTIPOLYGON (((-74.01244109299991 40.71905767...,0.000149,Battery Park City,13,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,43,41,0.052793,MULTIPOLYGON (((-73.94773985499985 40.80959972...,0.000143,Central Harlem,41,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,44,45,0.045907,MULTIPOLYGON (((-73.99750445299988 40.71406913...,0.000091,Chinatown,45,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,47,42,0.092709,MULTIPOLYGON (((-73.93436121591056 40.83620061...,0.000264,Central Harlem North,42,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,48,43,0.099739,MULTIPOLYGON (((-73.97255352299985 40.76489773...,0.000380,Central Park,43,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,52,48,0.043747,MULTIPOLYGON (((-73.99117738199989 40.75532984...,0.000094,Clinton East,48,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,54,50,0.055748,MULTIPOLYGON (((-73.99667705784736 40.77351005...,0.000173,Clinton West,50,Manhattan,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Manipulate Data

## Split into Hour and Day of the week

In [5]:
df_venue_merged['merged_time'] = pd.to_datetime(df_venue_merged['merged_time'])

# Add 'day_of_week' column (Monday as 0)
df_venue_merged['day_of_week'] = df_venue_merged['merged_time'].dt.dayofweek

# Add 'hour_integer' column
df_venue_merged['hour_integer'] = df_venue_merged['merged_time'].dt.hour

In [6]:
print(df_venue_merged.head(10))

   venue_id                   venue_name  busyness          venue_type  \
0  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
1  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
2  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
3  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
4  47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
5  47508476  Han Dynasty Upper West Side        20  CHINESE_RESTAURANT   
6  47508476  Han Dynasty Upper West Side        25  CHINESE_RESTAURANT   
7  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   
8  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   
9  47508476  Han Dynasty Upper West Side        30  CHINESE_RESTAURANT   

                                    venue_address         merged_time  \
0  215 W 85th St New York, NY 10024 United States 2023-06-19 06:00:00   
1  215 W 85th St New York, NY 10024 Uni

## Grouping Venue Types

In [7]:
venue_mapping = {
    'PARK': 'Park',
    'TOURIST_DESTINATION': 'Tourist Destination',
    'MUSEUM': 'Cultural Heritage',
    'HISTORICAL':'Cultural Heritage',
    'SCENIC_POINT': 'Scenic Landmarks',
    'BRIDGE': 'Scenic Landmarks',
    'NATURE_RESERVE': 'Nature Attractions',
    'ZOO': 'Nature Attractions',
    'BOTANICAL_GARDEN': 'Nature Attractions',
    'ARTS': 'Art',
    'DESSERT':'Art',
    'CHURCH': 'Religious',
    'SYNAGOGUE':'Religious',
    'VISITOR_CENTER': 'Tourist Destination',
    'LIBRARY':'Library',
    'SHOPPING_CENTER': 'Shopping Center',
    'APPAREL':'Fashion Convenience',
    'OTHER': 'Tourist Destination',
    'SHOPPING': 'Fashion Convenience',
    'CONVENIENCE_STORE':'Neighborhood Market',
    'SUPERMARKET': 'Neighborhood Market',
    'GROCERY':'Neighborhood Market',
    'MARKET':'Neighborhood Market',
    'GIFTS': 'Gifts & Souvenirs',
    'SOUVENIR_SHOP':'Gifts & Souvenirs',
    'CHINESE_RESTAURANT': 'Chinese Restaurant',
    'AMERICAN_RESTAURANT': 'American Restaurant',
    'RESTAURANT': 'Restaurant',
    'FRENCH_RESTAURANT': 'French Restaurant',
    'BREAKFAST_RESTAURANT': 'Breakfast Restaurant',
    'ITALIAN_RESTAURANT': 'Italian Restaurant',
    'MEDITERANEAN_RESTAURANT': 'Mediteranean Restaurant',
    'SEAFOOD_RESTAURANT': 'Seafood Restaurant',
    'JAPANESE_RESTAURANT': 'Japanese Restaurant',
    'THAI_RESTAURANT': 'Thai Restaurant',
    'VEGETERIAN_RESTAURANT': 'Vegeterian Restaurant',
    'MEXICAN_RESTAURANT': 'Mexican Restaurant',
    'PIZZA_RESTAURANT': 'Pizza Restaurant',
    'ASIAN_RESTAURANT': 'Asian Restaurant',
    'INDIAN_RESTAURANT': 'Indian Restaurant',
    'BURGER_RESTAURANT': 'Burger Restaurant',
    'SUSHI_RESTAURANT': 'Sushi Restaurant',
    'RAMEN_RESTAURANT': 'Ramen Restaurant',
}

df_venue_static['venue_mod_type'] = df_venue_static['venue_type'].replace(venue_mapping)

In [8]:
venue_to_zone_dict = {}

# Iterate through each row in the DataFrame
for index, row in df_venue_static.iterrows():
    venue_type = row['venue_mod_type']
    zone = row['zone_id']
    
    # If the venue_type is already in the dictionary, append the zone to its list
    if venue_type in venue_to_zone_dict:
        venue_to_zone_dict[venue_type].append(zone)
    # If the venue_type is not in the dictionary, create a new entry with the zone as a list
    else:
        venue_to_zone_dict[venue_type] = [zone]

print(venue_to_zone_dict)

{'Chinese Restaurant': [239, 239, 239, 239, 162, 239, 142], 'American Restaurant': [142, 263, 239, 79, 161, 100, 100, 239, 238, 243, 164, 249, 107, 236, 236, 142, 142, 148, 162, 48, 48, 234, 239, 238], 'Tourist Destination': [43, 170, 186, 161, 43, 166, 144, 239, 43, 230, 164, 43, 230, 43, 246, 162, 230, 50, 43, 43, 43, 161, 236, 170, 246], 'Restaurant': [230, 142, 162, 239, 239, 142, 48, 50, 239, 164, 239, 239, 48, 142, 239, 231, 230, 239, 239, 231, 48, 239, 239, 79, 239, 142, 239, 238, 263, 239, 68, 48, 142, 236, 142, 48, 238, 48, 239, 239, 141], 'French Restaurant': [161, 161, 239, 158, 230, 79, 237, 239, 161, 230], 'Breakfast Restaurant': [239, 239], 'Italian Restaurant': [238, 163, 237, 230, 238, 239, 142, 163, 239, 142, 142, 163, 142, 142, 249, 142, 238, 239, 239, 48, 142, 141, 249, 239, 142, 230, 114], 'Cultural Heritage': [43, 24, 163, 163, 236, 239, 103, 162, 239, 43, 239, 50, 170, 161, 236, 151, 43, 239, 158, 236, 43, 230, 100, 43, 238, 75, 142, 161, 75, 236, 43], 'Fashion Co

In [43]:
# df_venue_static.head(50)

# Clearing Duplicates

In [10]:
# look for duplicates
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_static.duplicated().sum())

# use "keep=False" to mark all duplicates as true, including the original rows that were duplicated
print('Number of duplicate rows (including first) in the table is:', df_venue_static[df_venue_static.duplicated(keep=False)].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 0


In [11]:
# look for duplicates
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_timings.duplicated().sum())

# use "keep=False" to mark all duplicates as true, including the original rows that were duplicated
print('Number of duplicate rows (including first) in the table is:', df_venue_timings[df_venue_timings.duplicated(subset=['venue_id', 'day', 'opening_time', 'closing_time'], keep='first')].shape[0])

Number of duplicate (excluding first) rows in the table is:  0
Number of duplicate rows (including first) in the table is: 777


In [12]:
print('Number of duplicate (excluding first) rows in the table is: ', df_venue_timings.drop_duplicates(subset=['venue_id', 'day', 'opening_time', 'closing_time'], inplace=True))
df_venue_timings

Number of duplicate (excluding first) rows in the table is:  None


,id,venue_id,day,opening_time,closing_time
0,1,ven_30356e77514c7172685648526b6f77595249473851...,0,11,22
1,2,ven_30356e77514c7172685648526b6f77595249473851...,1,11,22
2,3,ven_30356e77514c7172685648526b6f77595249473851...,2,11,22
3,4,ven_30356e77514c7172685648526b6f77595249473851...,3,11,22
4,5,ven_30356e77514c7172685648526b6f77595249473851...,4,11,22
...,...,...,...,...,...
3250,3629,ven_77446a687a695a67306746526b6f77595a6c644878...,2,8,0
3251,3630,ven_77446a687a695a67306746526b6f77595a6c644878...,3,8,0
3252,3631,ven_77446a687a695a67306746526b6f77595a6c644878...,4,8,0
3253,3632,ven_77446a687a695a67306746526b6f77595a6c644878...,5,8,1


# Grouping Zones

In [13]:
venue_zone_grouping = {
    'Upper Manhattan': [128, 127, 243, 120, 244, 116, 42, 152, 41, 74, 75],
    'Upper West Side': [166, 24, 151, 43, 238, 239, 143, 142],
    'Upper East Side': [236,263, 262, 237, 141, 140 ],
    'Chelsea/Greenwhich market':[246, 68, 186, 90, 100, 234, 158, 249, 113, 249],
    'Lower Manhattan': [107, 224, 114, 211, 144, 148, 232, 231, 45, 13, 261, 209, 87, 88, 12 ],
    'Midtown Manhattan': [50, 48, 230, 163, 161, 162, 229, 233, 164, 170, 137, 224, 107, 234]
}


In [14]:
# venue_zone_grouping dictionary
venue_zone_grouping = {
    'Upper Manhattan': [128, 127, 243, 120, 244, 116, 42, 152, 41, 74, 75],
    'Upper West Side': [166, 24, 151, 43, 238, 239, 143, 142],
    'Upper East Side': [236, 263, 262, 237, 141, 140],
    'Chelsea/Greenwhich market': [246, 68, 186, 90, 100, 234, 158, 249, 113, 249],
    'Lower Manhattan': [107, 224, 114, 211, 144, 148, 232, 231, 45, 13, 261, 209, 87, 88, 12],
    'Midtown Manhattan': [50, 48, 230, 163, 161, 162, 229, 233, 164, 170, 137, 224, 107, 234],
}

# Function to map zone numbers to zone groups
def map_zone_group(zone_number):
    for zone_group, zone_numbers in venue_zone_grouping.items():
        if zone_number in zone_numbers:
            return zone_group
    return 'Other'  # If zone number not found in the dictionary, assign 'Other'

# Create the 'zone_group' column based on the mapping
df_venue_static['zone_group'] = df_venue_static['zone_id'].apply(map_zone_group)

print(df_venue_static)


                                       original_ven_id  hash_ven_id  latitude  \
0    ven_30356e77514c7172685648526b6f77595249473851...     47508476   40.7875   
1    ven_30365f61684a68514c7730526b6f775a6c6b7a7072...     88409575   40.7770   
2    ven_303749644b67576e694b55526b6f7759685f4a4457...      1030056   40.7758   
3    ven_3038654c546f3936784873526b6f775a3148646771...     23854076   40.7580   
4    ven_3039793356676f745a3570526b6f775974496c6d73...     20848781   40.7776   
..                                                 ...          ...       ...   
351  ven_7768585a615270346e454b526b6f7766426c2d4f31...     40246733   40.7797   
352  ven_77686b76675f51577a7053526b6f775a356b526546...     76894533   40.7694   
353  ven_776c426b3172735a6f5455526b6f77595a5f4c7350...     96046596   40.7682   
354  ven_7774565736353150745f5f526b6f775a3949484637...     12087255   40.7244   
355  ven_7778775545424e486d7075526b6f7759465048596d...     31643001   40.7666   

     longitude  zone_id    

In [15]:
df_venue_static.to_csv('zone_Grouping.csv', index=False)

# Extracting Only Attraction Types and Ignoring Restaurants

In [16]:
unique_type_values = df_venue_static['venue_mod_type'].unique()
unique_type_values

array(['Chinese Restaurant', 'American Restaurant', 'Tourist Destination',
       'Restaurant', 'French Restaurant', 'Breakfast Restaurant',
       'Italian Restaurant', 'Cultural Heritage', 'Fashion Convenience',
       'Mediteranean Restaurant', 'BAR', 'Park', 'Neighborhood Market',
       'Shopping Center', 'Thai Restaurant', 'Scenic Landmarks',
       'Seafood Restaurant', 'Japanese Restaurant', 'Nature Attractions',
       'Gifts & Souvenirs', 'BEER', 'Vegeterian Restaurant',
       'Mexican Restaurant', 'Pizza Restaurant', 'FISHING', 'Art',
       'Asian Restaurant', 'Indian Restaurant', 'Burger Restaurant',
       'Religious', 'Sushi Restaurant', 'Library', 'Ramen Restaurant'],
      dtype=object)

In [17]:
specific_venue_types = ['Nature Attractions', 'Shopping Center', 'Tourist Destination', 'Cultural Heritage', 'Neighborhood Market', 'Fashion Convenience', 'Library', 'Scenic Landmarks', 'Art', 'Religious', 'Park', 'Gifts & Souvenirs']

# Filter the DataFrame to only include rows with the specific venue types
df_venue_static_att = df_venue_static[df_venue_static['venue_mod_type'].isin(specific_venue_types)]

# Now 'filtered_df' contains only rows where the "Attraction_Type" is in the specified list
print(df_venue_static_att)

                                       original_ven_id  hash_ven_id  latitude  \
2    ven_303749644b67576e694b55526b6f7759685f4a4457...      1030056   40.7758   
9    ven_304a6c77747a366c755455526b6f77594a70705759...     57103262   40.7739   
12   ven_30504d554d38614b6b6f78526b6f77597862677059...     91692844   40.7783   
17   ven_306c63565a476e6d385736526b6f775a39786b4a67...     54332763   40.7506   
18   ven_306d686f6679537162766c526b6f77324c69717a38...     44704230   40.8008   
..                                                 ...          ...       ...   
350  ven_7767787947374f4c683962526b6f775a3151357171...     39118662   40.7541   
351  ven_7768585a615270346e454b526b6f7766426c2d4f31...     40246733   40.7797   
353  ven_776c426b3172735a6f5455526b6f77595a5f4c7350...     96046596   40.7682   
354  ven_7774565736353150745f5f526b6f775a3949484637...     12087255   40.7244   
355  ven_7778775545424e486d7075526b6f7759465048596d...     31643001   40.7666   

     longitude  zone_id    

# Extracting Only Restaurants

In [18]:
restaurant_venue_types = ['Chinese Restaurant', 'American Restaurant', 'Restaurant', 'French Restaurant', 'Breakfast Restaurant', 'Italian Restaurant', 'Mediteranean Restaurant', 'Seafood Restaurant', 'Japanese Restaurant', 'Thai Restaurant', 'Vegeterian Restaurant', 'Mexican Restaurant', 'Pizza Restaurant', 'Asian Restaurant', 'Indian Restaurant', 'Burger Restaurant', 'Sushi Restaurant', 'Ramen Restaurant']

# Filter the DataFrame to only include rows with the specific venue types
df_venue_static_res = df_venue_static[df_venue_static['venue_mod_type'].isin(specific_venue_types)]

# Now 'filtered_df' contains only rows where the "Attraction_Type" is in the specified list
print(df_venue_static_res)

                                       original_ven_id  hash_ven_id  latitude  \
2    ven_303749644b67576e694b55526b6f7759685f4a4457...      1030056   40.7758   
9    ven_304a6c77747a366c755455526b6f77594a70705759...     57103262   40.7739   
12   ven_30504d554d38614b6b6f78526b6f77597862677059...     91692844   40.7783   
17   ven_306c63565a476e6d385736526b6f775a39786b4a67...     54332763   40.7506   
18   ven_306d686f6679537162766c526b6f77324c69717a38...     44704230   40.8008   
..                                                 ...          ...       ...   
350  ven_7767787947374f4c683962526b6f775a3151357171...     39118662   40.7541   
351  ven_7768585a615270346e454b526b6f7766426c2d4f31...     40246733   40.7797   
353  ven_776c426b3172735a6f5455526b6f77595a5f4c7350...     96046596   40.7682   
354  ven_7774565736353150745f5f526b6f775a3949484637...     12087255   40.7244   
355  ven_7778775545424e486d7075526b6f7759465048596d...     31643001   40.7666   

     longitude  zone_id    

# Actual Flow

### Input

In [19]:
user_zone_input = ['Upper Manhattan','Midtown Manhattan']

In [20]:
user_input_attractions = ['Fashion Convenience']

In [21]:
user_input_res_breakfast = ['Breakfast Restaurant']
user_input_res_lunch = ['Chinese Restaurant', 'French Restaurant']
user_input_res_dinner = ['American Restaurant', 'Sushi Restaurant']

In [22]:
# Create the suggested itinerary
current_date_time = datetime.now()

# Get the current day
user_input_day = current_date_time.weekday()

In [23]:
# Step 1: Filter the data to include only restaurants
df_restaurants = df_venue_static[df_venue_static['venue_type'].str.contains('restaurant', case=False)]

In [24]:
unique_type_values_att = df_venue_static_att['venue_mod_type'].unique()
unique_type_values_att

array(['Tourist Destination', 'Cultural Heritage', 'Fashion Convenience',
       'Park', 'Neighborhood Market', 'Shopping Center',
       'Scenic Landmarks', 'Nature Attractions', 'Gifts & Souvenirs',
       'Art', 'Religious', 'Library'], dtype=object)

In [25]:
max_attractions_per_day = 3

if len(user_input_attractions) < max_attractions_per_day:
    x = max_attractions_per_day - len(user_input_attractions)
    updated_list = [num for num in unique_type_values_att if num not in user_input_attractions]
    
    similarities = []
    user_input_tag_embedding = nlp(user_input_attractions[0]).vector

    for tag in updated_list:
        tag_embedding = nlp(tag).vector
        similarity = user_input_tag_embedding.dot(tag_embedding) / (np.linalg.norm(user_input_tag_embedding) * np.linalg.norm(tag_embedding))
        similarities.append(similarity)

    sorted_indices = np.argsort(similarities)[::-1]  # Descending order
    most_similar_tags = [updated_list[i] for i in sorted_indices]
    slice_most_similar_tags = most_similar_tags[0:x]
    user_input_attractions = user_input_attractions + slice_most_similar_tags

print(user_input_attractions)

['Fashion Convenience', 'Tourist Destination', 'Cultural Heritage']


In [26]:
# Get only the attractions relevant to the user
attractions_user_filtered = df_venue_static_att[df_venue_static_att['type_mod'].isin(user_input_attractions)]

In [27]:
user_input_attractions

['Fashion Convenience', 'Tourist Destination', 'Cultural Heritage']

In [28]:
def get_time_filtered_df(data_frame, day, open_time, close_time):

    # Loop through each venue type and check for the specified condition
    for column, venue_ids in data_frame.items():
        valid_venues = []
        if column == 'original_ven_id':
            for idx, venue_id in enumerate(venue_ids):
                venue_info = df_venue_timings.loc[df_venue_timings['venue_id'] == venue_id]
                filtered_venue_info = venue_info.loc[(venue_info['day'] == day) & (venue_info['opening_time'] <= open_time) 
                                                  & (venue_info['closing_time'] >= close_time)
                                                    ]
                if not filtered_venue_info.empty:
                    valid_venues.append(venue_id)
            break

    filtered_venues = data_frame[data_frame['original_ven_id'].isin(valid_venues)]
    
    return filtered_venues

In [29]:
# Assume that each attraction has a 2 hour visiting time + a 1 hour travelling time
attraction_visiting_time_hours = 3

# Divide the day into Breakfast, Lunch, Dinner and 3 attraction slot start times
attraction_slot_starts = [9, 13, 16]

# Extract the attractions relevant to each timeslot
attractions_per_slot = []
for slot in attraction_slot_starts:
    attractions_per_slot.append(get_time_filtered_df(attractions_user_filtered, user_input_day, slot, slot + attraction_visiting_time_hours))

# Extract the attraction types relevant to each timeslot
attractions_type_per_slot = []
for attraction_slot in attractions_per_slot:
    attractions_type_per_slot.append(attraction_slot['venue_mod_type'].unique())

print(attractions_type_per_slot)
print(attractions_type_per_slot[0])
print(attractions_type_per_slot[1])
print(attractions_type_per_slot[2])

[array(['Tourist Destination', 'Cultural Heritage', 'Fashion Convenience'],
      dtype=object), array(['Fashion Convenience', 'Cultural Heritage', 'Tourist Destination'],
      dtype=object), array(['Fashion Convenience', 'Tourist Destination', 'Cultural Heritage'],
      dtype=object)]
['Tourist Destination' 'Cultural Heritage' 'Fashion Convenience']
['Fashion Convenience' 'Cultural Heritage' 'Tourist Destination']
['Fashion Convenience' 'Tourist Destination' 'Cultural Heritage']


In [30]:
# Removes the specified attraction from the rows where it is not unique
def remove_non_unique_attraction(attraction_table, attraction_to_remove):
    
    # Update attraction slot copy
    for idx, rem_slot in enumerate(attraction_table):
        # Remove only if other options are available
        if (len(rem_slot) != 1) & (attraction_to_remove in rem_slot):
            attraction_table[idx] = [i for i in rem_slot if attraction_to_remove not in i]

    return attraction_table

In [31]:
# Copy structures for processing
user_attractions_copy          = user_input_attractions.copy()
attractions_type_per_slot_copy = attractions_type_per_slot.copy()

# Continue this loop until no element has been eliminated
element_eliminated = True
while element_eliminated :
    element_eliminated = False
    
    # If only one of the attractions are valid for a slot, schedule that attraction for the slot
    for attraction_slot in attractions_type_per_slot_copy:
        if (len(attraction_slot) == 1) & (attraction_slot[0] in user_attractions_copy):
            print('REMOVING ', attraction_slot[0])
            user_attractions_copy.remove(attraction_slot[0])
            element_eliminated = True
            attractions_type_per_slot_copy = remove_non_unique_attraction(attractions_type_per_slot_copy, attraction_slot[0])
        
            print(user_attractions_copy)
            print(attractions_type_per_slot_copy)

    # If no element was eliminated, go to the remaining slots and select random elements
    if not element_eliminated:
        for idx, attraction_slot in enumerate(attractions_type_per_slot_copy):
            if (len(attraction_slot) > 1):
                attractions_type_per_slot_copy[idx] = [i for i in attractions_type_per_slot_copy[idx] if attraction_slot[0] in i]
                print('REMOVING ', attraction_slot[0])
                user_attractions_copy.remove(attraction_slot[0])
                element_eliminated = True
                attractions_type_per_slot_copy = remove_non_unique_attraction(attractions_type_per_slot_copy, attraction_slot[0])
        
                print(user_attractions_copy)
                print(attractions_type_per_slot_copy)
                break

suggested_attraction_agenda = attractions_type_per_slot_copy

print('Unreachable Attractions: ', user_attractions_copy)
print('Suggested Agenda: ', attractions_type_per_slot_copy)

REMOVING  Tourist Destination
['Fashion Convenience', 'Cultural Heritage']
[['Tourist Destination'], ['Fashion Convenience', 'Cultural Heritage'], ['Fashion Convenience', 'Cultural Heritage']]
REMOVING  Fashion Convenience
['Cultural Heritage']
[['Tourist Destination'], ['Fashion Convenience'], ['Cultural Heritage']]
REMOVING  Cultural Heritage
[]
[['Tourist Destination'], ['Fashion Convenience'], ['Cultural Heritage']]
Unreachable Attractions:  []
Suggested Agenda:  [['Tourist Destination'], ['Fashion Convenience'], ['Cultural Heritage']]


In [32]:
zone_group = []
for group in user_zone_input:
    for zone in venue_zone_grouping[group]:
        zone_group.append(zone)
zone_group

[128,
 127,
 243,
 120,
 244,
 116,
 42,
 152,
 41,
 74,
 75,
 50,
 48,
 230,
 163,
 161,
 162,
 229,
 233,
 164,
 170,
 137,
 224,
 107,
 234]

In [33]:
# Filter the restaurant DataFrame based on the zone IDs
df_filtered_restaurants = df_restaurants[df_restaurants['zone_id'].isin(zone_group)]

# Print the filtered DataFrame
print(df_filtered_restaurants)

                                       original_ven_id  hash_ven_id  latitude  \
3    ven_3038654c546f3936784873526b6f775a3148646771...     23854076   40.7580   
5    ven_304330524e695f70503476526b6f775a6c6e5a7133...     43331760   40.7630   
8    ven_304961777454515336474c526b6f775a4e51737357...     55261692   40.7512   
15   ven_3068674e6f56796343637a526b6f77597431776642...     45765601   40.7639   
23   ven_342d4f33343573744c7a2d526b6f775a4277454754...      2779193   40.7545   
24   ven_34416b6a74734635334274526b6f775964314e4669...     61577324   40.7644   
45   ven_3855585f5861474e706f34526b6f77593930793336...     63592504   40.7646   
48   ven_385a743953644e70587637526b6f7759395f534f53...     75402097   40.7598   
66   ven_4148494f684649484f4477526b6f775a4e68634d42...     10162275   40.7457   
77   ven_4168437230416a45783158526b6f7759526c2d6f53...     19253272   40.7575   
80   ven_417239716752736e6c447a526b6f7759745078672d...     52953994   40.7611   
89   ven_454b716e73544e6e535

In [34]:
num_top_restaurants = 5  # just example 

# # Sort the DataFrame by rating in descending order and then by price in ascending order
# df_filtered_restaurants = df_filtered_restaurants.sort_values(by=['rating', 'price'], ascending=[False, True])

# # Display the top restaurants
# print(df_filtered_restaurants.head(num_top_restaurants))

In [35]:
print(df_filtered_restaurants)

                                       original_ven_id  hash_ven_id  latitude  \
3    ven_3038654c546f3936784873526b6f775a3148646771...     23854076   40.7580   
5    ven_304330524e695f70503476526b6f775a6c6e5a7133...     43331760   40.7630   
8    ven_304961777454515336474c526b6f775a4e51737357...     55261692   40.7512   
15   ven_3068674e6f56796343637a526b6f77597431776642...     45765601   40.7639   
23   ven_342d4f33343573744c7a2d526b6f775a4277454754...      2779193   40.7545   
24   ven_34416b6a74734635334274526b6f775964314e4669...     61577324   40.7644   
45   ven_3855585f5861474e706f34526b6f77593930793336...     63592504   40.7646   
48   ven_385a743953644e70587637526b6f7759395f534f53...     75402097   40.7598   
66   ven_4148494f684649484f4477526b6f775a4e68634d42...     10162275   40.7457   
77   ven_4168437230416a45783158526b6f7759526c2d6f53...     19253272   40.7575   
80   ven_417239716752736e6c447a526b6f7759745078672d...     52953994   40.7611   
89   ven_454b716e73544e6e535

In [36]:
print(df_venue_merged)

        venue_id                   venue_name  busyness          venue_type  \
0       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
1       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
2       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
3       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
4       47508476  Han Dynasty Upper West Side         0  CHINESE_RESTAURANT   
...          ...                          ...       ...                 ...   
231949  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   
231950  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   
231951  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   
231952  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   
231953  31643001     Hallett Nature Sanctuary         0      NATURE_RESERVE   

                                            venue_a

In [37]:
# Extract restaurant busyness
df_venue_merged['venue_id'] = df_venue_merged['venue_id'].astype(int)
column_length = df_filtered_restaurants.shape[0]
busyness_column = np.zeros(column_length)

test = True

for column, venue_ids in df_filtered_restaurants.items():
    if column == 'hash_ven_id':
        for idx, venue_hash_id in enumerate(venue_ids):
            specific_venue_df = df_venue_merged.loc[df_venue_merged['venue_id'] == venue_hash_id]
            busyness_column[idx] = specific_venue_df['busyness'].mean() 
        break

df_filtered_restaurants['busyness'] = busyness_column
print(df_filtered_restaurants['busyness'])

3       9.643939
5      16.212121
8      20.212121
15     14.098485
23     11.681818
24     52.462121
45     19.848485
48     20.628788
66     18.037879
77     29.371212
80     15.500000
89     20.303030
115    19.750000
142     7.401515
156     8.121212
159    12.234848
167    21.712121
170    27.272727
181    21.204545
184    24.409091
232    19.257576
270     8.113636
273    13.772727
285    17.553030
302    19.757576
307    23.666667
309    10.045455
319    17.431818
321    34.166667
322    22.492424
330    28.977273
334    19.333333
339    27.696970
Name: busyness, dtype: float64


In [38]:
introvert_mode = False  # We have it off by default?

# We normalise the price and rating columns to the range 0-1
df_filtered_restaurants['normalized_price'] = 1 - ((df_filtered_restaurants['price'] - df_filtered_restaurants['price'].min()) / (df_filtered_restaurants['price'].max() - df_filtered_restaurants['price'].min()))
df_filtered_restaurants['normalized_rating'] = (df_filtered_restaurants['rating'] - df_filtered_restaurants['rating'].min()) / (df_filtered_restaurants['rating'].max() - df_filtered_restaurants['rating'].min())

# We then define the weights for the rating and price. (These should add up to 1.)
rating_weight = 0.33
price_weight = 0.33
busyness_weight = 0.33

# We calculate the composite score here ? (if introvert mode is on, we take that into account)
df_filtered_restaurants['normalized_busyness'] = 1 - ((df_filtered_restaurants['busyness'] - df_filtered_restaurants['busyness'].min()) / (df_filtered_restaurants['busyness'].max() - df_filtered_restaurants['busyness'].min()))

if introvert_mode:
    df_filtered_restaurants['composite_score'] = df_filtered_restaurants['normalized_rating'] * rating_weight + df_filtered_restaurants['normalized_price'] * price_weight + df_filtered_restaurants['normalized_busyness'] * busyness_weight
else:
    df_filtered_restaurants['composite_score'] = df_filtered_restaurants['normalized_rating'] * rating_weight + df_filtered_restaurants['normalized_price'] * price_weight

# Compensate for user restaurant selections
df_filtered_restaurants['breakfast_score'] = (df_filtered_restaurants['venue_mod_type'].isin(user_input_res_breakfast)) * 0.5 + df_filtered_restaurants['composite_score'] * price_weight
df_filtered_restaurants['lunch_score'] = (df_filtered_restaurants['venue_mod_type'].isin(user_input_res_lunch    )) * 0.5 + df_filtered_restaurants['composite_score']     * price_weight
df_filtered_restaurants['dinner_score'] = (df_filtered_restaurants['venue_mod_type'].isin(user_input_res_dinner   )) * 0.5 + df_filtered_restaurants['composite_score']    * price_weight

# Then we can sort the DataFrame by the composite score in descending order
df_filtered_rest_breakfast = get_time_filtered_df(df_filtered_restaurants, user_input_day, 8, 9).sort_values('breakfast_score', ascending=False)
df_filtered_rest_lunch     = get_time_filtered_df(df_filtered_restaurants, user_input_day, 12, 13).sort_values('lunch_score', ascending=False)
df_filtered_rest_dinner    = get_time_filtered_df(df_filtered_restaurants, user_input_day, 19, 20).sort_values('dinner_score', ascending=False)

# Display the top restaurants
print('BREAKFAST: (Preferred ',user_input_res_breakfast,') ',df_filtered_rest_breakfast.head(num_top_restaurants))
print('LUNCH: (Preferred ',user_input_res_lunch,') ',df_filtered_rest_lunch.head(num_top_restaurants))
print('DINNER: (Preferred ',user_input_res_dinner,') ',df_filtered_rest_dinner.head(num_top_restaurants))

BREAKFAST: (Preferred  ['Breakfast Restaurant'] )                                         original_ven_id  hash_ven_id  latitude  \
170  ven_517362734b764763584752526b6f775a4a30324630...     85173724   40.7521   
15   ven_3068674e6f56796343637a526b6f77597431776642...     45765601   40.7639   

     longitude  zone_id                                               name  \
170   -73.9839      164                                  Harta & Bar Harta   
15    -73.9767      163  Mangia 57th - Midtown Italian Food & Corporate...   

                                           address           venue_type  \
170  30 W 39th St New York, NY 10018 United States  AMERICAN_RESTAURANT   
15   50 W 57th St New York, NY 10019 United States   ITALIAN_RESTAURANT   

     rating  price  ... location_id       venue_mod_type   busyness  \
170     4.3   -1.0  ...           0  American Restaurant  27.272727   
15      4.4    2.0  ...           0   Italian Restaurant  14.098485   

    normalized_price normalize

In [39]:
user_venue_per_type_dict = {}
for idx, venue_type in enumerate(suggested_attraction_agenda):
#     print(venue_type)
    start_time = attraction_slot_starts[idx]
    end_time = start_time + attraction_visiting_time_hours
    df_venue_static_att_filtered = get_time_filtered_df(df_venue_static_att, user_input_day, start_time, end_time)
    matched_zones = df_venue_static_att_filtered[df_venue_static_att_filtered['type_mod'] == venue_type[0]]['zone_id']
    matching_zones = matched_zones[matched_zones.isin(zone_group)]
    result_df = df_venue_static_att_filtered[df_venue_static_att_filtered['zone_id'].isin(matching_zones)]['original_ven_id']
    venue_ids = list(pd.DataFrame(result_df, columns=['original_ven_id'])['original_ven_id'])
    
#     # Store the list of venue IDs as the value in the dictionary
    user_venue_per_type_dict[venue_type[0]] = venue_ids
# Print the resulting dictionary
print(user_venue_per_type_dict)


{'Tourist Destination': ['ven_51695f51454843725a5431526b6f775952656f495459354a496843', 'ven_555234376f4f4f30364447526b6f775a4a7752465057654a496843', 'ven_556242455a4d50646d7736526b6f775a7061734749664e4a496843', 'ven_556e5f3675393756787575526b6f775a4a674f497a396a4a496843', 'ven_63436e304a565f4d627257526b6f7759745667317257724a496843', 'ven_67664d4c43655043527777526b6f7759785f7774527a304a496843', 'ven_7732374951527a41425143526b6f7759706c584c4859394a496843'], 'Fashion Convenience': ['ven_344d7478767a4b74377548526b6f7759566c78494a516d4a496843', 'ven_38584b356562656e4a4a63526b6f77595a5f4d6d7456674a496843', 'ven_3866497051714e61656e4b526b6f7759683141556f5a544a496843', 'ven_386b4c712d792d70556e33526b6f77596c5f66384652324a496843', 'ven_4537554f41513330705337526b6f7759705f6736675a764a496843', 'ven_4565635872326c5643414d526b6f775a70364b4e5452304a496843', 'ven_456c694e4c624c6f555862526b6f7759566c74633152704a496843', 'ven_494a68323156454c57744d526b6f775935666f68472d764a496843', 'ven_4d566a675754566

In [41]:
user_restaurant_per_zone_dict = {}
for venue_type in user_input_attractions:
    matched_zones = df_restaurants[df_restaurants['venue_mod_type'] == venue_type]['zone_id']
    matching_zones = matched_zones[matched_zones.isin(zone_group)]
    result_df = df_restaurants[df_restaurants['zone_id'].isin(matching_zones)]['original_ven_id']
    user_restaurant_per_zone_dict[venue_type] = list(result_df)

In [ ]:
# Need to check for empty Type

In [42]:
zone_type_dict = {}
for venue_type in itinerary_timing:
    matched_zones = df_venue_static_att[df_venue_static_att['venue_mod_type'] == venue_type]['zone_id'].unique()
    zone_type_dict[venue_type] = list(matched_zones)

# Print the resulting dictionary
print(zone_type_dict)

NameError: name 'itinerary_timing' is not defined

In [ ]:
types_with_zero_zones = []

# Iterate through the venue_type_dict
for venue_type, zones in zone_type_dict.items():
    if len(zones) == 0:
        types_with_zero_zones.append(venue_type)

print("Venue types with 0 zones:", types_with_zero_zones)

In [ ]:
def find_distance_between_zones(zone1_polygon, zone2_polygon):
    # Find the nearest points between the two polygons
    nearest_points_result = nearest_points(wkt.loads(zone1_polygon), wkt.loads(zone2_polygon))

    # Calculate the distance between the nearest points
    distance = nearest_points_result[0].distance(nearest_points_result[1])
    return distance

In [ ]:
filled_type_with_zero_zone = {}

if len(types_with_zero_zones) > 0:
    for ven_type in types_with_zero_zones:
        venue_to_zone_dict_copy = list(set(venue_to_zone_dict[i]))
        print(lenvenue_to_zone_dict_copy)
        if len(venue_to_zone_dict_copy) < 0:
            #get all venue id of each zone and push it to user_venue_per_type_dict of that type
            print('okay')
        else:
            zone_between_dist = []
            for user_zone in zone_group:
                for venue_zone in venue_to_zone_dict_copy:
                    zone1_polygon = df_manhattan_zone[df_manhattan_zone['LocationID'] == user_zone]['the_geom'].iloc[0]
                    zone2_polygon = df_manhattan_zone[df_manhattan_zone['LocationID'] == venue_zone]['the_geom'].iloc[0]
                    distance = find_distance_between_zones(zone1_polygon, zone2_polygon)
                    zone_between_dist.append((user_zone, venue_zone, distance)) 
            sorted_zone_between_dist = sorted(zone_between_dist, key=lambda x: x[2])[:3]
            new_zone = [df_venue_static_att[
        (df_venue_static_att['zone_id'] == item[1]) &
        (df_venue_static_att['venue_mod_type'] == ven_type)
    ]['original_ven_id'].tolist() for item in sorted_zone_between_dist]
    
    print(new_zone)
    
    filled_type_with_zero_zone[ven_type] = list(set(item for sublist in new_zone for item in sublist))
    filled_type_with_zero_zone

In [ ]:
for key in user_venue_per_type_dict.keys():
    # Check if the value of the current key is an empty array
    if len(user_venue_per_type_dict[key]) == 0:
        # Check if the key exists in dictionary B
        if key in filled_type_with_zero_zone:
            # Replace the value in A with the value from B
            user_venue_per_type_dict[key] = filled_type_with_zero_zone[key]
print(user_venue_per_type_dict)

In [ ]:
print(user_venue_per_type_dict)

In [ ]:
time_index = 0
filtered_venues = {}

# Loop through each venue type and check for the specified condition
for venue_type, venue_ids in user_venue_per_type_dict.items():
    valid_venues = []
    for venue_id in venue_ids:
        venue_info = df_venue_timings.loc[df_venue_timings['venue_id'] == venue_id]
        filtered_venue_info = venue_info.loc[(venue_info['day'] == user_input_day) & (venue_info['opening_time'] <= opening_time[time_index]) 
                                              & (venue_info['closing_time'] >= closing_time[time_index])
                                            ]
        if not filtered_venue_info.empty:
            valid_venues.append(venue_id)
    if valid_venues:
        filtered_venues[venue_type] = valid_venues
        time_index+=1
print(filtered_venues)


In [ ]:
manipulated_venues = {}

if introvert_mode:
    weight_rating = 0.3
    weight_busyness = 0.7
else:
    weight_rating = 0.6
    weight_busyness = 0.4

# With introvert mode
for venue_type, venue_ids in user_venue_per_type_dict.items():
    valid_venues = []
    for venue_id in venue_ids:
        venue_hash_id = df_venue_static.loc[df_venue_static['original_ven_id'] == venue_id]['hash_ven_id']
        venue_rating = df_venue_static.loc[df_venue_static['original_ven_id'] == venue_id]['rating'].item()
        venue_hash_id = int(venue_hash_id)
        df_venue_merged['venue_id'] = df_venue_merged['venue_id'].astype(int)
        specific_venue_df = df_venue_merged.loc[df_venue_merged['venue_id'] == venue_hash_id]
        average_busyness = specific_venue_df['busyness'].mean() 
        
        composite_score = (weight_rating * venue_rating) + (weight_busyness * average_busyness)
        
        valid_venues.append((venue_id, venue_rating, average_busyness, composite_score))
    if valid_venues:
        manipulated_venues[venue_type] = valid_venues

manipulated_venues

In [ ]:
def print_top_3_recommended_restaurants(data_frame):
    for rank, ind in enumerate(data_frame.index):
        venue_id = data_frame['original_ven_id'][ind]
        rating = data_frame['rating'][ind]
        busyness = data_frame['busyness'][ind]
        score = data_frame['composite_score'][ind]
        print(f"Rank {rank + 1}: Venue ID: {venue_id}, Rating: {rating}, Busyness: {busyness}, Score: {score}")
        if rank > 1:
            break;

In [ ]:
top_3_venues = {}

# Loop through each venue type and its venues
for venue_type, venue_data in manipulated_venues.items():
    # Sort the venues based on the composite score (fourth element in the tuple, index 3)
    if len(venue_data) > 3:
        sorted_venues = sorted(venue_data, key=lambda x: x[3], reverse=True)
    
        # Keep only the top 3 venues for each venue type
        top_3_venues[venue_type] = sorted_venues[:3]
    else:
        top_3_venues[venue_type] = venue_data

# Display the top 3 venues for each venue type
for venue_type, top_venues in top_3_venues.items():
    print(f"Venue Type: {venue_type}")
    for rank, (venue_id, rating, busyness, score) in enumerate(top_venues, start=1):
        print(f"Rank {rank}: Venue ID: {venue_id}, Rating: {rating}, Busyness: {busyness}, Score: {score}")
    print()

# Display the top 3 restaurants for each type
print(f"Restaurant Type: Breakfast")
print_top_3_recommended_restaurants(df_filtered_rest_breakfast)
print()
print(f"Restaurant Type: Lunch")
print_top_3_recommended_restaurants(df_filtered_rest_lunch)
print()
print(f"Restaurant Type: Dinner")
print_top_3_recommended_restaurants(df_filtered_rest_dinner)

print()